## lendo os dados do hycom

In [88]:
import numpy as np
import pandas as pd
from glob import glob
from netCDF4 import Dataset

In [89]:
cd '/home/gus/Documentos/mestrado/dados-mestrado/dados-mercator'

/home/gus/Documentos/mestrado/dados-mestrado/dados-mercator


In [90]:
#reading data
data = Dataset('mercator_dado_validacao.nc')

In [91]:
data.dimensions

{'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 293,
 'depth': <class 'netCDF4._netCDF4.Dimension'>: name = 'depth', size = 19,
 'latitude': <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 37,
 'longitude': <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 37}

In [92]:
data.variables

{'depth': <class 'netCDF4._netCDF4.Variable'>
 float32 depth(depth)
     valid_min: 0.494025
     valid_max: 55.76429
     units: m
     positive: down
     unit_long: Meters
     long_name: Depth
     standard_name: depth
     axis: Z
     _ChunkSizes: 50
     _CoordinateAxisType: Height
     _CoordinateZisPositive: down
 unlimited dimensions: 
 current shape = (19,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'latitude': <class 'netCDF4._netCDF4.Variable'>
 float32 latitude(latitude)
     valid_min: -5.0
     valid_max: -2.0
     step: 0.08333588
     units: degrees_north
     unit_long: Degrees North
     long_name: Latitude
     standard_name: latitude
     axis: Y
     _ChunkSizes: 2041
     _CoordinateAxisType: Lat
 unlimited dimensions: 
 current shape = (37,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'vo': <class 'netCDF4._netCDF4.Variable'>
 int16 vo(time, depth, latitude, longitude)
     long_name: Northward velocity
     standard_nam

In [93]:
def cartesian(x, y):

    # This function creates a matrix with two columns, 
    # the first referring to the x axis and the second to the y axis. 
    # Each row in the matrix contains a pair (x, y).

    from numpy import zeros
    global pairs
    pairs = zeros ([len(x) * len(y), 2])

    counter = 0

    for line in range (len(x)):

        for colunm in range(len(y)):

            pairs [counter, 0] = x [line]
            pairs [counter, 1] = y [colunm]
            counter += 1

    return pairs

In [94]:
def closest(array, x0, y0):

    distances = ((array[:,0]-x0)**2\
        + (array[:,1]-y0)**2)**.5
    d_min = distances.min()

    lon_point= array[distances == d_min, 0]

    lat_point = array[distances == d_min, 1]

    return lon_point, lat_point

In [95]:
def lon360to180(lon):
    """
    Converts longitude values in the range [0,360]
    to longitude values in the range [-180,+180].
    """
    lon = np.asanyarray(lon)
    
    return ((lon + 180.) % 360.) - 180

In [96]:
def hours2dates(reference_date, values):
    """"
    Converts hours values in datetime based in a
    specific date.  The reference date must be in the
    format Day/Month/Year Hour:Minutes
    """
    import datetime as dt
    
    date_python = dt.datetime.strptime(reference_date,
                                  '%d/%m/%Y %H:%M')
    
    Time = [date_python + dt.timedelta(hours = x) for x in values]
    
    return Time

In [97]:
#getting the values lon, lat and time

lat = data['latitude'][:].squeeze()

lon = data['longitude'][:].squeeze()

depth = data['depth'][:].squeeze()

#lon = lon360to180(lon)

Time = data['time'][:]

TimeDatetime = hours2dates('01/01/1950 00:00',Time)

In [98]:
min(depth), max(depth)

(0.494025, 55.76429)

In [99]:
lon.min(), lon.max()

(-40.0, -37.0)

In [100]:
array = cartesian(lon, lat);
coordinates = closest(array, -38.5, -3.17)

In [101]:
lon_point = coordinates[0].squeeze()
lat_point = coordinates[1].squeeze()

In [102]:
lon_point, lat_point

(array(-38.5), array(-3.16666675))

In [103]:
np.savetxt('coordenadas_mercator.csv',pairs,'%.2f', delimiter=';')

In [104]:
list(enumerate(depth))

[(0, 0.494025),
 (1, 1.541375),
 (2, 2.645669),
 (3, 3.819495),
 (4, 5.078224),
 (5, 6.440614),
 (6, 7.92956),
 (7, 9.572997),
 (8, 11.405),
 (9, 13.46714),
 (10, 15.81007),
 (11, 18.49556),
 (12, 21.59882),
 (13, 25.21141),
 (14, 29.44473),
 (15, 34.43415),
 (16, 40.34405),
 (17, 47.37369),
 (18, 55.76429)]

In [105]:
whos

Variable       Type           Data/Info
---------------------------------------
CompU          DataFrame                  8m        16m<...>n\n[293 rows x 5 columns]
CompV          DataFrame                  8m        16m<...>n\n[293 rows x 5 columns]
Dataset        type           <class 'netCDF4._netCDF4.Dataset'>
Time           MaskedArray    [586596. 586620. 586644. <...> 593556. 593580. 593604.]
TimeDatetime   list           n=293
array          ndarray        1369x2: 2738 elems, type `float64`, 21904 bytes
cartesian      function       <function cartesian at 0x7f7e58d6e550>
closest        function       <function closest at 0x7f7e58d86dc0>
coordinates    tuple          n=2
data           Dataset        <class 'netCDF4._netCDF4.<...>(longitude)\n    groups: 
depth          MaskedArray    [ 0.494025  1.541375  2.6<...>405  47.37369  55.76429 ]
glob           function       <function glob at 0x7f7ec41d81f0>
hours2dates    function       <function hours2dates at 0x7f7e58170ee0>
lat   

In [106]:
zlevels = [6, 10, 14, 16]
zdepths = ['8m', '16m','32m','40m']

In [107]:
CompV = data['vo'][:, zlevels,lat==lat_point,
                   lon==lon_point].squeeze()
CompU = data['uo'][:, zlevels,lat==lat_point,
                   lon==lon_point].squeeze()

In [108]:
CompV.shape, CompU.shape

((293, 4), (293, 4))

In [109]:
TimeDatetime[0], TimeDatetime[-1]

(datetime.datetime(2016, 12, 1, 12, 0), datetime.datetime(2017, 9, 19, 12, 0))

In [110]:
CompU *= 100;
CompV *= 100;

## salvando os dados

In [111]:
CompU = pd.DataFrame(CompU);
CompV = pd.DataFrame(CompV);

In [112]:
CompU.columns = zdepths
CompV.columns = zdepths

In [113]:
CompU.head()

,8m,16m,32m,40m
0,-43.397320,-41.566210,-40.284432,-39.124729
1,-45.655690,-44.190802,-43.458357,-43.214209
2,-42.664876,-41.199988,-40.650655,-40.650655
3,-36.255989,-34.607990,-34.058657,-34.119694
4,-37.049470,-35.462508,-34.791101,-34.730064


In [114]:
CompU['time'] = TimeDatetime
CompV['time'] = TimeDatetime

In [115]:
CompU.to_csv('mercator_zonal_2011a2019.csv', sep=';', index=False)
CompV.to_csv('mercator_meridional_2011a2019.csv', sep=';', index=False)